# Final

The final class project is to develop a model to detect pulmonary infection (pneumonia) on chest radiographs using any of the approaches and tools you have learned this quarter. The goal is both to create a high-performing algorithm for the target task, as well as to analyze performance across several different architecture permutations. At minimum, three different network designs of your choice will be tested (you are welcome to include more if you've tested others). As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for **all** models (each independently saved)
* final compiled `*.csv` file with performance statistics across the different architectures
* final write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [1]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.1 library. Use the following line of code to select this updated version:

# Environment

In [3]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x
% pip install tensorflow-gpu==2.1

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [4]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries (note that depending on architecture choices, various additional modules will need to be specified here):

In [0]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics, regularizers
from jarvis.train import datasets, custom

# Data

The data used in this tutorial will consist of (frontal projection) chest radiographs from the RSNA / Kaggle pneumonia challenge (https://www.kaggle.com/c/rsna-pneumonia-detection-challenge). The chest radiograph is the standard screening exam of choice to identify and trend changes in lung disease including infection (pneumonia). 

### Download

The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/xr_pna`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [6]:
# --- Download dataset
datasets.download(name='xr/pna-crp')

{'code': '/data/raw/xr_pna', 'data': '/data/raw/xr_pna'}

### Python generators

Once the dataset is downloaded locally, Python generators to iterate through the dataset can be easily prepared using the `datasets.prepare(...)` method.

In [0]:
# --- Prepare generators
gen_train, gen_valid, client = datasets.prepare(name='xr/pna-crp', keyword='crp')

The created generators yield two variables for each iteration, `xs` and `ys`, each representing a dictionary of model input(s) and output(s).

### Model inputs

For every input in `xs`, a corresponding `Input(...)` variable can be created and returned in a `inputs` dictionary for ease of model development:

In [8]:
# --- Create model inputs
inputs = client.get_inputs(Input)

print(inputs.keys())
print(inputs['dat'].shape)
print(inputs['msk'].shape)

dict_keys(['dat', 'msk'])
(None, 1, 256, 128, 1)
(None, 1, 256, 128, 1)


# Training

The goal of this project is to perform **global classification** for each image. In other words, regardless of algorithm choice, the final objective is to determine the absence or presence of pneumonia for each sample. This does **not** mean that you are required to use classification networks only; in fact it very well may be the case that a localization algorithm will overall perform better on this task.

The task is designed to be open-ended on purpose. The only requirements are to:

* test at minimum three different network architectures
* one algorithm must use (at least) a classification type loss function
* one algorithm must use (at least) a segmentation type loss function

Note that the qualifier *at least* indicates that if you choose, you can use both classification and segmentation losses simultaneously for one algorithm to satisfy both requirements (this would allow you to test only non-classification / non-segmentation architectures for all remaining models if you choose).

#Classification Model

In [0]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

In [0]:
# --- Define model with extra initial layers
l1 = conv1(32, conv1(32, inputs['dat']))
l2 = conv1(48, conv1(48, conv2(48, l1)))
l3 = conv1(64, conv1(64, conv2(64, l2)))
l4 = conv1(80, conv1(80, conv2(80, l3)))
l5 = conv1(96, conv1(96, conv2(96, l4)))
l6 = conv1(112, conv1(112, conv2(112, l5)))

# --- Flatten / reshape
c1 = layers.Reshape((-1, 1, 1, 8 * 4 * 112))(l6)

# --- Create logits
logits = {}
logits['pna-cls'] = layers.Conv3D(filters=2, kernel_size=(1, 1, 1), name='pna-cls')(c1)

# --- Create model
model_cls = Model(inputs=inputs, outputs=logits) 

# --- Compile the model
model_cls.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'pna-cls': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={'pna-cls': metrics.SparseCategoricalAccuracy()},
    experimental_run_tf_function=False)

In [9]:
# --- Load data into memory for faster training
client.load_data_in_memory()

# --- Train the model
model_cls.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

[ 2020-06-11 10:31:58 ] [====================] 100.000% : Iterating | 029694    Epoch 1/12
250/250 [==============================] - 44s 177ms/step - loss: 0.5312 - sparse_categorical_accuracy: 0.7870
Epoch 2/12
250/250 [==============================] - 32s 130ms/step - loss: 0.4454 - sparse_categorical_accuracy: 0.8203
Epoch 3/12
250/250 [==============================] - 32s 129ms/step - loss: 0.3896 - sparse_categorical_accuracy: 0.8287
Epoch 4/12
250/250 [==============================] - 45s 179ms/step - loss: 0.3933 - sparse_categorical_accuracy: 0.8320 - val_loss: 0.4285 - val_sparse_categorical_accuracy: 0.8273
Epoch 5/12
250/250 [==============================] - 32s 129ms/step - loss: 0.3860 - sparse_categorical_accuracy: 0.8393
Epoch 6/12
250/250 [==============================] - 32s 129ms/step - loss: 0.3550 - sparse_categorical_accuracy: 0.8493
Epoch 7/12
250/250 [==============================] - 32s 129ms/step - loss: 0.3457 - sparse_categorical_accuracy: 0.8463
Epoch

Evaluation

In [0]:
fname = '{}/models/final/model_cls.hdf5'.format(MOUNT_ROOT)
model_cls = models.load_model(fname, compile=False)

In [0]:
def calculate_stats(pred, true):
    """
    Method to calculate binary classification stats
    
    """
    # --- Calculate true positives / true negatives
    tp = np.count_nonzero((true == 1) & (pred == 1))
    tn = np.count_nonzero((true == 0) & (pred == 0))
    fp = np.count_nonzero((true == 0) & (pred == 1))
    fn = np.count_nonzero((true == 1) & (pred == 0))

    # --- Stats
    acc = (tp + tn) / (tp + tn + fp + fn)
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    
    return acc, sen, spe, ppv, npv

In [11]:
# --- Run model prediction for segmentation masks
test_train, test_valid = client.create_generators(test=True)

pred = []
true = []

for xs, ys in test_train:
    
    logs = model_cls.predict(xs)
    cls_ = np.argmax(logs, axis=-1)
    
    # --- Record number of pixels in mask
    pred.append(cls_.squeeze())
    true.append(ys['pna-cls'].squeeze())

    if len(pred) == 1000:
        break

# --- Convert to arrays
cls_tr_pred = np.array(pred)
cls_tr_true = np.array(true)
cls_tr_mean = calculate_stats(cls_tr_pred, cls_tr_true)[0]
print (cls_tr_mean)

[ 2020-06-11 15:22:36 ] [>...................] 4.200% : Iterating | 001000      0.876


In [12]:
pred = []
true = []

for xs, ys in test_valid:
    
    logs = model_cls.predict(xs)
    cls_ = np.argmax(logs, axis=-1)
    
    # --- Record number of pixels in mask
    pred.append(cls_.squeeze())
    true.append(ys['pna-cls'].squeeze())

    if len(pred) == 1000:
        break

# --- Convert to arrays
cls_val_pred = np.array(pred)
cls_val_true = np.array(true)
cls_val_mean = calculate_stats(cls_val_pred, cls_val_true)[0]
print (cls_val_mean)

[ 2020-06-11 15:23:13 ] [===>................] 16.992% : Iterating | 001000     0.863


#Segmentation Model

In [0]:
# --- Select shape
configs = {'specs': {'xs': {'dat': {'shape': [3, 256, 128, 1]}}}, 
           'batch': {'size': 12}}

# --- Cropped 256 x 128 (multiple step)
gen_train, gen_valid, client = datasets.prepare(name='xr/pna-crp', keyword='crp', configs=configs)
inputs = client.get_inputs(Input)

In [0]:
# --- Define 2D conv (xy-features)
conv_2d = lambda x, filters, strides : layers.Conv3D(
    filters=filters, 
    strides=strides, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define 1D conv (z-features)
conv_1d = lambda x, filters, k=2 : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(k, 1, 1),
    padding='valid',
    kernel_initializer='he_normal')(x)

# --- Define stride-1 3D, stride-2 3D and stride-1 1D (z-subsample) blocks
conv1 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 1, 1))))
conv2 = lambda filters, x : relu(norm(conv_2d(x, filters, strides=(1, 2, 2))))
convZ = lambda filters, k, x : relu(norm(conv_1d(x, filters, k=k)))

# --- Define 2D transpose
tran = lambda x, filters : layers.Conv3DTranspose(
    filters=filters, 
    strides=(1, 2, 2),
    kernel_size=(1, 3, 3),
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define transpose block
tran2 = lambda filters, x : relu(norm(tran(x, filters)))

In [0]:
# --- Define model
l1 = conv1(32,  inputs['dat'])
l2 = conv1(48, conv2(48, l1))
l3 = conv1(64, conv2(64, l2))
l4 = conv1(80, convZ(80, 2, conv2(80, l3)))
l5 = conv1(96, convZ(96, 2, conv2(96, l4)))
l6 =  tran2(80, conv1(80, l5))
l7 =  tran2(64, conv1(80, convZ(80, 2, l4) + l6))
l8 =  tran2(48, conv1(64, convZ(64, 3, l3) + l7))
l9 =  tran2(32,  conv1(48, convZ(48, 3, l2) + l8))
l10 = conv1(32,  conv1(32,  convZ(32, 3, l1) + l9))


# --- Create logits
logits_custom = {}
logits_custom['pna-seg'] = layers.Conv3D(
    name='pna-seg',
    filters=3, 
    strides=1, 
    kernel_size=(1, 3, 3), 
    padding='same',
    kernel_initializer='he_normal')(l10)

# --- Create model
model_seg = Model(inputs=inputs, outputs=logits_custom)

In [32]:
# --- Create metrics
metrics = custom.dsc(weights=inputs['msk'])
metrics += [custom.softmax_ce_sens(weights=inputs['msk'])]
metrics = {'pna-seg': metrics}

# --- Compile the model
model_seg.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'pna-seg': custom.sce(inputs['msk'])},
    metrics=metrics,
    experimental_run_tf_function=False)

# --- Load data into memory for faster training
client.load_data_in_memory()

# --- Train the model
model_seg.fit(
    x=gen_train, 
    steps_per_epoch=250, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=250,
    validation_freq=4)

[ 2020-06-11 11:59:55 ] [====================] 100.000% : Iterating | 029694    Epoch 1/12
250/250 [==============================] - 113s 451ms/step - loss: 0.1261 - dsc_1: 0.4012 - softmax_ce_sens: 0.3145
Epoch 2/12
250/250 [==============================] - 106s 422ms/step - loss: 0.0964 - dsc_1: 0.5153 - softmax_ce_sens: 0.4696
Epoch 3/12
250/250 [==============================] - 105s 422ms/step - loss: 0.0905 - dsc_1: 0.5442 - softmax_ce_sens: 0.5018
Epoch 4/12
250/250 [==============================] - 142s 568ms/step - loss: 0.0922 - dsc_1: 0.5442 - softmax_ce_sens: 0.5030 - val_loss: 0.0957 - val_dsc_1: 0.5121 - val_softmax_ce_sens: 0.4577
Epoch 5/12
250/250 [==============================] - 106s 422ms/step - loss: 0.0833 - dsc_1: 0.5527 - softmax_ce_sens: 0.5181
Epoch 6/12
250/250 [==============================] - 106s 423ms/step - loss: 0.0851 - dsc_1: 0.5970 - softmax_ce_sens: 0.5619
Epoch 7/12
250/250 [==============================] - 106s 422ms/step - loss: 0.0853 - ds

Evaluation

In [0]:
fname = '{}/models/final/model_seg.hdf5'.format(MOUNT_ROOT)
model_seg = models.load_model(fname, compile=False)

In [0]:
def calculate_stats(nnzs, true, threshold):
    """
    Method to calculate stats at given threshold of mask pixels
    
    """
    # --- Calculate true positives / true negatives
    tp = np.count_nonzero(nnzs[true == 1] >= threshold)
    tn = np.count_nonzero(nnzs[true == 0] < threshold)
    fp = np.count_nonzero(nnzs[true == 0] >= threshold)
    fn = np.count_nonzero(nnzs[true == 1] < threshold)

    # --- Stats
    acc = (tp + tn) / (tp + tn + fp + fn)
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    
    return acc, sen, spe, ppv, npv

In [16]:
# --- Run model prediction for segmentation masks
test_train, test_valid = client.create_generators(test=True)

nnzs = []
true = []

for xs, ys in test_train:
    
    logs = model_seg.predict(xs)
    pred = np.argmax(logs, axis=-1)
    
    # --- Remove masked pixels if needed
    pred[xs['msk'][..., 0] == 0] = 0
    
    # --- Record number of pixels in mask
    nnzs.append(np.count_nonzero(pred))
    true.append(ys['pna-cls'].squeeze())
    
    # --- Break after 1000 exams
    if len(nnzs) == 1000:
        break

# --- Convert to arrays
seg_tr_nnzs = np.array(nnzs)
seg_tr_true = np.array(true)
seg_tr_mean = calculate_stats(seg_tr_nnzs, seg_tr_true, 1000)[0]
print (seg_tr_mean)

[ 2020-06-11 15:23:59 ] [>...................] 4.200% : Iterating | 001000      

In [17]:
nnzs = []
true = []

for xs, ys in test_valid:
    
    logs = model_seg.predict(xs)
    pred = np.argmax(logs, axis=-1)
    
    # --- Remove masked pixels if needed
    pred[xs['msk'][..., 0] == 0] = 0
    
    # --- Record number of pixels in mask
    nnzs.append(np.count_nonzero(pred))
    true.append(ys['pna-cls'].squeeze())
    
    # --- Break after 1000 exams
    if len(nnzs) == 1000:
        break

# --- Convert to arrays
seg_val_nnzs = np.array(nnzs)
seg_val_true = np.array(true)
seg_val_mean = calculate_stats(seg_val_nnzs, seg_val_true, 1000)[0]
print (seg_val_mean)

[ 2020-06-11 15:24:41 ] [===>................] 16.992% : Iterating | 001000     0.884


#Dual loss function Model

In [0]:
from jarvis.utils.general import overload, tools as jtools
from jarvis.train.client import Client

@overload(Client)
def preprocess(self, arrays, **kwargs):
    """
    Method to create a custom msk array for class weights and/or masks
    
    """
    # --- Create msk
    msk = np.zeros(arrays['xs']['dat'].shape)

    lng = arrays['xs']['msk']
    pna = arrays['ys']['pna-seg']

    msk[lng > 0] = 1
    msk[pna > 0] = 1

    arrays['xs']['msk'] = msk
    
    return arrays

In [0]:
# --- Prepare generators
configs = {'batch': {'size': 8}}
gen_train, gen_valid, client = datasets.prepare(name='xr/pna-crp', keyword='crp', configs=configs)

# --- Manually create generators
gen_train, gen_valid = client.create_generators()

# --- Create model inputs
inputs = client.get_inputs(Input)

In [0]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same',
    'kernel_regularizer': regularizers.l2(0.01)}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

# --- Define dropout
drop = layers.Dropout(rate=0.25)

In [0]:
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(80, conv2(80, l5))
l6_conv = drop(conv1(96, conv2(96, l6)))


# --- Define expanding layers
l6_tran = tran2(80, l6_conv)
l7  = tran2(64, conv1(80, l6_tran  + l6))
l8  = tran2(48, conv1(64, l7  + l5))
l9  = tran2(32, conv1(48, l8  + l4))
l10 = tran2(16, conv1(32, l9  + l3))
l11 = tran2(8,  conv1(16, l10 + l2))
l12 = conv1(8,  conv1(8,  l11 + l1))

# --- Create classifier feature vector
c1 = layers.Reshape((-1, 1, 1, 4 * 2 * 96))(l6_conv)

# --- Create logits
logits = {}
logits['pna-seg'] = layers.Conv3D(filters=2, name='pna-seg', **kwargs)(l12)
logits['pna-cls'] = layers.Conv3D(filters=2, kernel_size=(1, 1, 1), name='pna-cls')(c1)

# --- Create model
model_duo = Model(inputs=inputs, outputs=logits) 

In [0]:
# --- Compile the model
model_duo.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'pna-seg': custom.sce(inputs['msk']),
        'pna-cls': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={
        'pna-seg': custom.dsc(weights=inputs['msk']),
        'pna-cls': metrics.SparseCategoricalAccuracy()
        },
    experimental_run_tf_function=False)

In [19]:
# --- Load data into memory for faster training
client.load_data_in_memory()

# --- Train model
model_duo.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=12,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=4,
    use_multiprocessing=True)

[ 2020-06-11 14:19:59 ] [====================] 100.000% : Iterating | 029694    WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.
Epoch 1/12
100/100 [==============================] - 15s 153ms/step - loss: 10.8002 - pna-cls_loss: 0.6255 - pna-seg_loss: 0.2264 - pna-cls_sparse_categorical_accuracy: 0.7387 - pna-seg_dsc_1: 0.3752
Epoch 2/12
100/100 [==============================] - 7s 69ms/step - loss: 8.5053 - pna-cls_loss: 0.5832 - pna-seg_loss: 0.1499 - pna-cls_sparse_categorical_accuracy: 0.7600 - pna-seg_dsc_1: 0.4143
Epoch 3/12
100/100 [==============================] - 7s 67ms/step - loss: 7.4186 - pna-cls_loss: 0.5325 - pna-seg_loss: 0.1251 - pna-cls_sparse_categorical_accuracy: 0.7700 - pna-seg_dsc_1: 0.5264
Epoch 4/12
 99/100 [============================>.] - ETA: 0s - loss: 6.8221 - pna-cls_loss: 0.4753 - pna-seg_loss: 0.1207 - pna-cls_sparse_categorical_accu

Evaluation

In [0]:
fname = '{}/models/final/model_duo.hdf5'.format(MOUNT_ROOT)
model_duo = models.load_model(fname, compile=False)

In [0]:
def calculate_stats(nnzs, true, threshold):
    """
    Method to calculate stats at given threshold of mask pixels
    
    """
    # --- Calculate true positives / true negatives
    tp = np.count_nonzero(nnzs[true == 1] >= threshold)
    tn = np.count_nonzero(nnzs[true == 0] < threshold)
    fp = np.count_nonzero(nnzs[true == 0] >= threshold)
    fn = np.count_nonzero(nnzs[true == 1] < threshold)

    # --- Stats
    acc = (tp + tn) / (tp + tn + fp + fn)
    sen = tp / (tp + fn)
    spe = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    
    return acc, sen, spe, ppv, npv

In [22]:
# --- Run model prediction for segmentation masks
test_train, test_valid = client.create_generators(test=True)

nnzs = []
true = []

for xs, ys in test_train:
    
    logits = model_duo.predict(xs)
    pred = np.argmax(logits[1], axis=-1)
     
    # --- Remove masked pixels if needed
    pred[xs['msk'][..., 0] == 0] = 0
    
    # --- Record number of pixels in mask
    nnzs.append(np.count_nonzero(pred))
    true.append(ys['pna-cls'].squeeze())
    
    # --- Break after 1000 exams
    if len(nnzs) == 1000:
        break

duo_tr_nnzs = np.array(nnzs)
duo_tr_true = np.array(true)

duo_tr_mean = calculate_stats(duo_tr_nnzs, duo_tr_true, 1000)[0]
print (duo_tr_mean)

[ 2020-06-11 15:25:21 ] [>...................] 4.200% : Iterating | 001000      0.867


In [23]:
nnzs = []
true = []

for xs, ys in test_valid:
    
    logits = model_duo.predict(xs)
    pred = np.argmax(logits[1], axis=-1)
    
    # --- Remove masked pixels if needed
    pred[xs['msk'][..., 0] == 0] = 0
    
    # --- Record number of pixels in mask
    nnzs.append(np.count_nonzero(pred))
    true.append(ys['pna-cls'].squeeze())
    
    # --- Break after 1000 exams
    if len(nnzs) == 1000:
        break

duo_val_nnzs = np.array(nnzs)
duo_val_true = np.array(true)

duo_val_mean = calculate_stats(duo_val_nnzs, duo_val_true, 1000)[0]
print (duo_val_mean)

[ 2020-06-11 15:26:01 ] [===>................] 16.992% : Iterating | 001000     0.85


# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* overall accuracy (mean)
* overall sensitivity
* overall specificity
* overall positive predictive value (PPV)
* overall negative predictive value (NPV)

### Performance

The only requirement for full credit is that your overall top-performing model achieves an image-by-image accuracy rate of 0.85 or above. In addition, the **top three performing models** out of the entire class will recieve a full letter bonus to your overall final grade (e.g. C to B, B to A, etc). 

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          ACCURACY                    
          mean | median | 25th-tile | 75th-tile 
model 1
model 2
model 3
...
```

As above, tables for both training and validation should be provided.

In [0]:

# --- Create *.csv
data_train = {'tr_mean':[cls_tr_mean, seg_tr_mean, duo_tr_mean],
         'tr_median':[np.median(cls_tr_pred), np.median(seg_tr_nnzs), np.median(duo_tr_nnzs)],
         'tr_25th-tile':[np.percentile(cls_tr_pred,25),np.percentile(seg_tr_nnzs,25), np.percentile(duo_tr_nnzs,25)],
         'tr_75th-tile':[np.percentile(cls_tr_pred,75),np.percentile(seg_tr_nnzs,75), np.percentile(duo_tr_nnzs,75)]     
         }
df_train = pd.DataFrame(data_train, index=['model 1', 'model 2', 'model 3'])                             


In [34]:
print("Training cohort statistics")
df_train

Training cohort statistics


,tr_mean,tr_median,tr_25th-tile,tr_75th-tile
model 1,0.876,0.0,0.0,1.00
model 2,0.886,81.0,0.0,5044.75
model 3,0.867,314.0,9.0,4145.75


In [0]:
data_valid = {'val_mean':[cls_val_mean, seg_val_mean, duo_val_mean],
         'val_median':[np.median(cls_val_pred), np.median(seg_val_nnzs), np.median(duo_val_nnzs)],
         'val_25th-tile':[np.percentile(cls_val_pred,25),np.percentile(seg_val_nnzs,25), np.percentile(duo_val_nnzs,25)],
         'val_75th-tile':[np.percentile(cls_val_pred,75),np.percentile(seg_val_nnzs,75), np.percentile(duo_val_nnzs,75)]     
         }
df_valid = pd.DataFrame(data_valid, index=['model 1', 'model 2', 'model 3'])   

In [36]:
print("Validation cohort statistics")
df_valid

Validation cohort statistics


,val_mean,val_median,val_25th-tile,val_75th-tile
model 1,0.863,0.0,0.0,1.0
model 2,0.884,0.0,0.0,4009.0
model 3,0.850,189.0,7.0,2984.0


In [37]:
result = pd.concat([df_train, df_valid.reindex(df_train.index)], axis=1)
result

,tr_mean,tr_median,tr_25th-tile,tr_75th-tile,val_mean,val_median,val_25th-tile,val_75th-tile
model 1,0.876,0.0,0.0,1.00,0.863,0.0,0.0,1.0
model 2,0.886,81.0,0.0,5044.75,0.884,0.0,0.0,4009.0
model 3,0.867,314.0,9.0,4145.75,0.850,189.0,7.0,2984.0


In [0]:
# --- Serialize *.csv
fname = '{}/models/final/gej4_results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
result.to_csv(fname)

# Summary

In addition to algorithm training as above, a brief write-up is required for this project (minimum of one page). The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? Which loss types were most effective for this task (e.g. classification, segmentation, etc)? With more time and/or resources, how would further optimize your top model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.

In [0]:
fname = '{}/models/final/model_seg.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model_seg.save(fname)

fname = '{}/models/final/model_cls.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model_cls.save(fname)

fname = '{}/models/final/model_duo.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model_duo.save(fname)